# Assignment 1

Using text http://www.gutenberg.org/files/2600/2600-0.txt
1. Make text lowercase and remove all punctuation except spaces and dots.
2. Tokenize text by BPE with vocab_size = 100
3. Train 3-gram language model with laplace smoothing $\delta=1$
4. Using beam search with k=10 generate sequences of length=10 conditioned on provided inputs. Treat dots as terminal tokens.
5. Calculate perplexity of the language model for the first sentence.

In [2]:
from google.colab import drive                  #настройка доступа к файлам на Google drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [19]:
text = open('/content/gdrive/My Drive/2600-0.txt', 'r').read()[2:]
len(text)

3227579

In [20]:
import string
import re

def preprocess_text(text):
    text_lower = text.lower()
    clean_text = re.sub(r'[^\w\d\. ]+', '', text_lower)
    no_spaces_text = re.sub("  +",' ',clean_text)
    return no_spaces_text

text = preprocess_text(text)
len(text)
#assert len(text) == 3141169

3078785

In [0]:
text = text.split('.')
text = [x.strip() for x in text[:1000]]

In [0]:
print(text)

['the project gutenberg ebook of war and peace by leo tolstoythis ebook is for the use of anyone anywhere at no cost and with almostno restrictions whatsoever', 'you may copy it give it away or reuseit under the terms of the project gutenberg license included with thisebook or online at www', 'gutenberg', 'orgtitle war and peaceauthor leo tolstoytranslators louise and aylmer maudeposting date january 10 2009 ebook 2600last updated january 21 2019language englishcharacter set encoding utf8 start of this project gutenberg ebook war and peace an anonymous volunteer and david widgerwar and peaceby leo tolstoytolstoi contents book one 1805 chapter i chapter ii chapter iii chapter iv chapter v chapter vi chapter vii chapter viii chapter ix chapter x chapter xi chapter xii chapter xiii chapter xiv chapter xv chapter xvi chapter xvii chapter xviii chapter xix chapter xx chapter xxi chapter xxii chapter xxiii chapter xxiv chapter xxv chapter xxvi chapter xxvii chapter xxviii book two 1805 chapt

In [0]:
def flatten(list):
  for i in list:
    for j in i:
      yield j

In [0]:
from collections import Counter
import nltk
from sklearn.base import TransformerMixin


class BPE(TransformerMixin):
    def __init__(self, vocab_size=100):
        super(BPE, self).__init__()
        self.vocab_size = vocab_size
        # index to token
        self.itos = []
        # token to index
        self.stoi = {}
        
    def fit(self, text):
        """
        fit itos and stoi
        text: list of strings 
        """
        
        # TODO
        # tokenize text by symbols and fill in self.itos and self.stoi
        text = [list(sentence) for sentence in text]
        #text = list(flatten(text))
        self.itos = list(set(list(flatten(text))))
        for i, item in enumerate(self.itos):
          self.stoi[item] = i
        
        text = [[self.stoi[sym] for sym in sent] for sent in text]
        
        while len(self.itos) < self.vocab_size:
            # TODO
            # count bigram freqencies in the text
            bigram_frq = Counter(zip(list(flatten(text)), list(flatten(text))[1:]))
            new_token = bigram_frq.most_common(1)[0][0]
            # most common bigram in the text
            new_id = len(self.itos)
            
            self.itos.append(new_token)
            self.stoi[new_token] = new_id
            
            
            # find occurences of the new_token in the text and replace them with new_id
            for sent in text:
              indices = [i for i, x in enumerate(sent) if x == new_token[0]]
              indices_to_remove = []

              for ind in indices:
                if ind!=len(sent)-1:
                  if sent[ind+1] == new_token[1]:
                    indices_to_remove.append(ind+1)
                    sent[ind] = new_id
              
              sent = [i for j, i in enumerate(sent) if j not in indices_to_remove]
            
        return self
    
    def transform(self, text):
        """
        convert text to a sequence of token ids
        text: list of strings
        """
        # TODO tokenize text by symbols using self.stoi
        text = [list(sentence) for sentence in text]
        #text = list(flatten(text))
        text = [[self.stoi[sym] for sym in sent] for sent in text]
        
        for sent in text:
          for el in range(len(self.itos)):
            indices = [i for i, x in enumerate(sent) if x == el]
  
            needed_itos = [k for k in self.itos if type(k)!=str and k[0] == el]
          #print(el, needed_itos)
            for ix in indices:
              if ix<len(sent)-1:
                for item in needed_itos:
                  if sent[ix+1] == item[1]:
                    sent = sent[:ix]+[self.stoi[item]]+sent[ix+2:] 
                  
        return text
      
    def decode_token(self, tok):
        """
        tok: int or tuple
        """
        result = []
        if type(tok) is int:
          result = self.itos[tok]
        else:
          for k in tok:
            if type(self.itos[k]) == str:
              result.append(k)
            else:
              new = self.decode_token(k)
              result += new
            
        #result = [self.itos[int(num)] for num in result]
        return result
            
    def decode(self, text):
        """
        convert token ids into text
        """
        return ''.join(map(self.decode_token, text))
        
  
  
    
vocab_size = 100
bpe = BPE(vocab_size)
tokenized_text = bpe.fit_transform(text)

In [0]:
assert bpe.decode(tokenized_text[0]) == text[0]

In [0]:
import numpy as np
from itertools import tee, islice


start_token = vocab_size
end_token = vocab_size + 1

bpe.itos.append('@')
bpe.itos.append(' ')

bpe.stoi['@'] = start_token
bpe.stoi['.'] = end_token

def ngrams(lst, n):
  tlst = lst
  while True:
    a, b = tee(tlst)
    l = tuple(islice(a, n))
    if len(l) == n:
      yield l
      next(b)
      tlst = b
    else:
      break
        
    
class LM:
    def __init__(self, vocab_size, delta=1):
        self.delta = delta
        self.vocab_size = vocab_size + 2
        self.proba = [] # TODO create array for storing 3-gram counters
        
    def fit(self, text):
        """
        train language model on text
        text: list of lists
        """
        
        text = [[start_token]+sent+[end_token] for sent in text]
        text = list(flatten(text))
        self.proba = Counter(ngrams(text, 3))
        return self
  
    def get_proba(self, a, b, c, tau=1):
        """
        get probability of 3-gram (a,b,c)
        a: first token id
        b: second token id
        c: third token id
        tau: temperature
        
        #вероятност триграммы абс
        """
        counter_result = self.proba
        count_current_3gram = counter_result[(a,  b, c)]
        count_all_3grams = 0
        for count in counter_result:
          if count[0]==a and count[1]==b:
            count_all_3grams += counter_result[count]
        result = ((count_current_3gram + self.delta) ** (1/tau))/((count_all_3grams + self.delta) ** (1/tau))
        # TODO approximate probability by counters
        return result
    
    def infer(self, a, b, tau=1):
        """
        return vector of probabilities of size self.vocab for 3-grams which start with (a,b) tokens
        ## вектор длины 100 (размер вокаба) в котором хранятся вероятности всех триграмм , начинающихся с а b
        a: first token id
        b: second token id
        tau: temperature
        """
        vector = [] #[0] * len(self.vocab)
        for item in range(self.vocab_size):
          vector.append(self.get_proba(a, b, item, tau=1))
        result = vector
        #print(result)
        return result
      
    
    
lm = LM(vocab_size, 1).fit(tokenized_text)

In [0]:
def beam_search(input_seq, lm, max_len=10, k=5, tau=1):
    """
    generate sequence from language model *lm* conditioned on input_seq
    input_seq: sequence of token ids for conditioning
    lm: language model
    max_len: max generated sequence length
    k: size of beam
    tau: temperature
    """

    beam = [(input_seq, 1)]
    
    # TODO store in beam tuples of current sequences and their log probabilities
    
    for i in range(max_len-len(input_seq)): ###тут минус уже имеющееся наверное??????):
      #print ('BEAM', beam)
      candidates = []
      candidates_proba = [] #LM(vocab_size, 1).infer(a, b, tau=1)
    
      for item in beam:
          #print(item)
          snt = item[0]
          snt_proba = item[1]
          #print(type(snt))
          #print(snt[-1], snt_proba)
          x = snt[-2]
          y = snt[-1]
          #print(a,b)
          if y == end_token:
            continue
          else:    
              proba = lm.infer(x, y, tau) # probability vector of the next token
              #print(len(proba))
              sorted_probs = sorted([(e,i) for i,e in enumerate(proba)], reverse = True)
              #print(len(bpe.itos))
              #print(sorted_probs)
              best_k = [bpe.itos[ix[1]] for ix in sorted_probs[:k]]# top-k most probable tokens
              
              for token in best_k:
                candidates.append(item[0]+[bpe.stoi[token]])
                candidates_proba.append(sorted_probs[bpe.stoi[token]][0])
                # TODO update candidates' sequences and corresponding probabilities
                
      beam = [(candidates[i],candidates_proba[i]) for i in range(k)]
      
    return beam
    

In [36]:

input1 = 'horse '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities

for item in result:
  print(bpe.decode(item[0]), item[1])
    

horse and  0.0010787486515641855
horse andr 0.006472491909385114
horse ande 0.003236245954692557
horse ands 0.0010787486515641855
horse ando 0.0010787486515641855
horse andi 0.007551240560949299
horse and  0.0010787486515641855
horse andt 0.0010787486515641855
horse andl 0.007551240560949299
horse anda 0.6796116504854369


In [33]:
input1 = 'her'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities
print(result)

for item in result:
  print(bpe.decode(item[0]), item[1])
    

[([15, 19, 13, 21, 36, 12, 12, 21, 46, 15], 0.001199040767386091), ([15, 19, 13, 21, 36, 12, 12, 21, 46, 22], 0.001199040767386091), ([15, 19, 13, 21, 36, 12, 12, 21, 46, 0], 0.5323741007194245), ([15, 19, 13, 21, 36, 12, 12, 21, 46, 11], 0.001199040767386091), ([15, 19, 13, 21, 36, 12, 12, 21, 46, 13], 0.001199040767386091), ([15, 19, 13, 21, 36, 12, 12, 21, 46, 27], 0.001199040767386091), ([15, 19, 13, 21, 36, 12, 12, 21, 46, 19], 0.001199040767386091), ([15, 19, 13, 21, 36, 12, 12, 21, 46, 12], 0.001199040767386091), ([15, 19, 13, 21, 36, 12, 12, 21, 46, 39], 0.001199040767386091), ([15, 19, 13, 21, 36, 12, 12, 21, 46, 21], 0.001199040767386091)]
her xii ch 0.001199040767386091
her xii co 0.001199040767386091
her xii ca 0.5323741007194245
her xii cl 0.001199040767386091
her xii cr 0.001199040767386091
her xii cu 0.001199040767386091
her xii ce 0.001199040767386091
her xii ci 0.001199040767386091
her xii cy 0.001199040767386091
her xii c  0.001199040767386091


In [31]:
input1 = 'what'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=1)
# TODO print decoded generated strings and their probabilities


for item in result:
  print(bpe.decode(item[0]), item[1])
    

what the a 0.12210526315789473
what the t 0.0003508771929824561
what the w 0.0007017543859649122
what the s 0.0007017543859649122
what the h 0.02912280701754386
what the o 0.004561403508771929
what the i 0.03368421052631579
what the p 0.0003508771929824561
what the c 0.0003508771929824561
what the b 0.0003508771929824561


In [32]:
input1 = 'gun '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities
for item in result:
  print(bpe.decode(item[0]), item[1])
    

gun the an 0.0005367686527106817
gun the a  0.0005367686527106817
gun the as 0.0005367686527106817
gun the al 0.015029522275899088
gun the at 0.0005367686527106817
gun the ar 0.00912506709608159
gun the ab 0.0005367686527106817
gun the am 0.008588298443370907
gun the ag 0.0005367686527106817
gun the af 0.0005367686527106817


In [45]:
from math import log

def perplexity(snt, lm):
    """
    snt: sequence of token ids
    lm: language model
    """
    text = list(snt)
    counter_result = Counter(ngrams(text, 3))
    
    trigrams_probs = [log(lm.get_proba(k[0],k[1],k[2], tau=1)) for k in counter_result.keys()]
    m = len(snt)
    result = 2 **(-1/m*(sum(trigrams_probs)))
    return result

perplexity(tokenized_text[0], lm)

3.383703965630975